In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import os
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle
import imp

# from pcanet import PCANet

%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# set cwd back to default
os.chdir('../')
os.getcwd()

'/Users/mark/src/dog-breed-identification/src'

In [3]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'validation' = ['img_name']
    , 'labels' = pandas.df('img_name','label')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [4]:
import data # src/data.py
dataset = data.init_dataset()

### Select the amount of classes that will be used

In [5]:
# pick the n classes with the most occuring instances
amt = 3
classes = data.top_classes(dataset.labels, amt)
classes

['paper', 'glass', 'plastic']

In [6]:
maxx = 10
max_train = 10
x_train, n = data.extract_topx_classes(dataset, classes, 'train', maxx, max_train)
x_test, _ = data.extract_topx_classes(dataset, classes, 'test', maxx, max_train)
x_validation, _ = data.extract_topx_classes(dataset, classes, 'validation', maxx, max_train)
print(n)

10


In [7]:
for a in [x_train,x_test,x_validation]:
    print(len(a))

30
30
30


### Load and convert images

Training data

In [8]:
x_train, y_train, n = data.extract_all(dataset, x_train, dirname='train/')

extract all data: 30


Testing data

In [9]:
x_test, y_test, n = data.extract_all(dataset, x_test, dirname='test/')

extract all data: 30


Validation data

In [10]:
x_validation, y_validation, n  = data.extract_all(dataset, x_validation, dirname='validation/')

extract all data: 30


In [11]:
# x_validation

Convert labels and determine input and output shape for model

In [12]:
y_train, y_test, y_validation = data.labels_to_vectors(dataset, y_train, y_test, y_validation)
input_shape = x_train.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label
print(output_length)

6


Initiate VG16 network and add extra layers

In [ ]:
from keras.models import Model 

def get_model(learn_rate=0.01, dropout=0.10, epochs=2, batch_size=5):
    # build the VGG16 network
    # global input_shape
    model = applications.VGG16(weights='imagenet', include_top=False, input_shape = input_shape )
    print('Model loaded, n layers:',  len(model.layers))


    for layer in model.layers[0:15]:
        layer.trainable = False

    #Adding custom Layers 
    x = model.output
    x = Flatten()(x)
    x = (Dense(128, activation='relu'))(x) #128
    x = (Dropout(dropout))(x)
    x = (Dense(16, activation='relu'))(x)
    # softmax output to get probability distribution
    predictions = Dense(output_length, activation="softmax")(x)


    # creating the final model 
    model_final = Model(input = model.input, output = predictions)

    # Adam, SGD
    # sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
    optimizer = optimizers.Adam(lr=learn_rate)

    # compile the model 
    model_final.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy',
        'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])
    
    return model_final

### Hyperparameter selection

In [ ]:
from sklearn.grid_search import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import time

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# batches = [5, 10, 20]
# learn_rate = [0.0001, 0.001, 0.01]
# dropout = [0.05, 0.1, 0.25] # most used values
param_grid = {'learn_rate':[0.01,0.001,0.001],'dropout':[0.05,0.1,0.25]}
param_grid = {'learn_rate':[0.001,0.001]} # ,'dropout':[0.05,0.1,0.25]}

# n = len(param_grid.values()) + 1
n =1

# tune the hyperparameters via a randomized search
model = KerasClassifier(build_fn=get_model) # verbose=0
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=n, n_jobs=1)
# watch memory usage

In [15]:
start = time.time()
grid.fit(x_train, y_train)

# evaluate the best randomized searched model on the testing
# data
print("\n[INFO] randomized search took {:.2f} seconds".format(time.time() - start))

Model loaded.


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/1
10/10 [==============================] - 19s 2s/step
Model loaded.
Epoch 1/1
10/10 [==============================] - 18s 2s/step
Model loaded.
Epoch 1/1
10/10 [==============================] - 20s 2s/step
Model loaded.
Epoch 1/1
30/30 [==============================] - 140s 5s/step - loss: 8.8702 - acc: 0.1667 - mean_squared_error: 0.2454 - categorical_crossentropy: 8.8702 - top_k_categorical_accuracy: 0.8333
[INFO] randomized search took 365.14 seconds
30/30 [==============================] - 133s 4s/step
[INFO] grid search accuracy: 36.67%
[INFO] randomized search best parameters: {'learn_rate': 0.001}


In [ ]:
acc = grid.score(x_validation, y_validation)
print("\n[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(
    grid.best_params_))

## Data augmentation
(to increase the 'quality' of the data)

In [16]:
#Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

### Train the model

In [17]:
# n epochs = n iterations over all the training data
epochs = 1 # 50

### With data augmentation

In [18]:
# train_datagen.fit(x_train)

# model_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
#                     steps_per_epoch=len(x_train) / batch_size, epochs=epochs, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])
        


### Without data augmentation

In [19]:
# from keras.callbacks import TensorBoard
# model_final.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
#           validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

In [20]:
# model_final = get_model()
# model_final.summary()

### running tests

In [21]:
# cvscores = []
# scores = model.evaluate(x_test, y_test, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# cvscores.append(scores[1] * 100)
# print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

### Baseline

In [22]:
# from pcanet import PCANet

# pcanet = PCANet(
#         image_shape= (512,348),
#         filter_shape_l1=2, step_shape_l1=1, n_l1_output=4,
#         filter_shape_l2=2, step_shape_l2=1, n_l2_output=4,
#         block_shape=2
#     )
# pcanet.validate_structure()

# pcanet.fit(x_train)
# X_train = pcanet.transform(x_train)
# X_test = pcanet.transform(x_test)

# model = RandomForestClassifier(n_estimators=100, random_state=1234, n_jobs=-1)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("accuracy: " + str(accuracy))
